### Input data

In [46]:
import gurobipy as gb
import numpy as np
import pandas as pd

gen_costs = pd.read_excel('gen_costs.xlsx')
gen_data = pd.read_excel('gen_data.xlsx')
line_data = pd.read_excel('line_data.xlsx')
system_demand = pd.read_excel('system_demand.xlsx')
load_distribution = pd.read_excel('load_distribution.xlsx')

wind_data = []
for i in range(6):
    wind_data.append(pd.read_csv('wind 1.out'))


n_bus = 24

bus_data = pd.DataFrame(index = np.arange(n_bus), data = np.zeros(n_bus), columns = ['Load'])
bus_data['Wind'] = 0

line_data['Susceptance pu'] = 1 / line_data['Reactance pu']

branch_matrix = np.zeros((n_bus, n_bus)) #susceptance

#Filling out susceptance matrix

for n in range(1, n_bus + 1):
    branch_matrix[n - 1][n - 1] = line_data.loc[(line_data['From'] == n) | (line_data['To'] == n), 'Susceptance pu'].sum()
    for k in range(n, n_bus + 1): #start from n
        if k != n:
            branch_matrix[n - 1][k - 1] = -1 * (line_data.loc[(line_data['From'] == n) & (line_data['To'] == k), 'Susceptance pu'].sum())
            branch_matrix[k - 1][n - 1] = branch_matrix[n - 1][k - 1]

<div>
<img src="image.png" width="700"/>
</div>


### Task 1 (single time step)

In [14]:
#Choosing to run for the first hour
load = np.zeros(n_bus)
demand = system_demand['System Demand'][0]

for n in load_distribution['Node'].unique():
    load[n] = load_distribution.loc[load_distribution['Node'] == n, '% of system load'] / 100 * demand

load

array([  0.      ,  67.48173 ,  60.37839 , 111.877605,  46.17171 ,
        44.395875,  85.24008 ,  78.13674 , 106.5501  , 108.325935,
       120.75678 ,   0.      ,   0.      , 165.152655, 120.75678 ,
       197.117685,  62.154225,   0.      , 207.772695, 113.65344 ,
        79.912575,   0.      ,   0.      ,   0.      ])

In [12]:
#per-unit bases
S_base_3ph = 100 #MW

direction = gb.GRB.MINIMIZE #Min / Max

# Create a Gurobi model       
m = gb.Model()

# Add variables
p_G = m.addVars(len(gen_data.index), lb=0, ub=gb.GRB.INFINITY, name="P_G") #Note: This is in per unit
theta = m.addVars(n_bus, lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="theta")

# Set objective function
obj = gb.quicksum(gen_costs['C ($/MWh)'][k] * p_G[k] * S_base_3ph for k in range(len(p_G)))         
m.setObjective(obj, direction) #Minimization / Maximization



#m.addConstrs(p_G[i] - demands[i] == theta[i] * b_mat.iloc[i,i] + gb.quicksum(theta[k] * b_mat.iloc[i,k] for k in range(n_bus) if k != i) for i in range(n_bus))

# for i in range(n_bus):
#     for k in range(i, n_bus): #Avoid duplicates
#         if (i != k) and (branch_reactances[i][k] != 0):
#             m.addConstr((theta[i] - theta[k]) * 1 / branch_reactances[i][k] <= branch_flow_limit[i][k])
#             m.addConstr((theta[i] - theta[k]) * 1 / branch_reactances[i][k] >= -1 * branch_flow_limit[i][k])

# m.addConstrs(p_G[i] <= capacities[i] for i in range(n_bus))
# m.addConstr(theta[0] == 0)

m.update()
m.display()

Minimize
1332.0 P_G[0] + 1332.0 P_G[1] + 2070.0 P_G[2] + 2093.0 P_G[3] + 2611.0 P_G[4]
+ 1052.0 P_G[5] + 1052.0 P_G[6] + 602.0 P_G[7] + 547.0 P_G[8] + 1052.0 P_G[10]
+ 1089.0 P_G[11]
Subject To
Bounds
  theta[0] free
  theta[1] free
  theta[2] free
  theta[3] free
  theta[4] free
  theta[5] free
  theta[6] free
  theta[7] free
  theta[8] free
  theta[9] free
  theta[10] free
  theta[11] free
  theta[12] free
  theta[13] free
  theta[14] free
  theta[15] free
  theta[16] free
  theta[17] free
  theta[18] free
  theta[19] free
  theta[20] free
  theta[21] free
  theta[22] free
  theta[23] free


Run model (NOT UPDATED)

In [81]:
m.optimize()

# Check if the optimization was successful and print solutions
if m.status == gb.GRB.OPTIMAL:
    constraints = m.getConstrs()
    # The constraint dual value / sensitivity in the current solution (also known as the shadow price)... https://www.gurobi.com/documentation/9.5/refman/pi.html
    optimized_sens = [constraints[k].Pi for k in range(len(constraints))] 
    print('-----------------------------------------------')
    print("Optimal objective value: %.2f DKK" % m.objVal)

    for i in range(n_bus):
        print(p_G[i].VarName + ": %.2f pu" % p_G[i].x)
        print(theta[i].VarName + ": %.2f rad" % theta[i].x)

    for k in range(len(constraints)):
        print('Dual value {0}: '.format(k+1), optimized_sens[k])

    print("\nFlows:")
    for i in range(n_bus):
        for k in range(i, n_bus):
            if (i != k) and (branch_reactances[i][k] != 0):
                print('From %d to %d: %.2f pu' % (i + 1, k + 1, (theta[i].x - theta[k].x) * 1 / branch_reactances[i][k]))
    for i in range(n_bus):
        net_flow = 0
        for k in range(n_bus):
            if (i != k) and (branch_reactances[i][k] != 0):
                net_flow += (theta[i].x - theta[k].x) * 1 / branch_reactances[i][k]
        print('Bus %d net injection: %.2f pu' % (i + 1, net_flow))

    print("\nTotal load: %.2f MWh" % (sum(demands[n] for n in range(n_bus)) * S_base_3ph))
    print("Total generation: %.2f MWh" % (sum(p_G[n].x for n in range(n_bus)) * S_base_3ph))

else:
    print("Optimization was not successful.")     
  
        
# Dispose of the Gurobi model to release resources
m.dispose()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 3700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 6 columns and 28 nonzeros
Model fingerprint: 0x9cb6c417
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+03, 2e+04]
  Bounds range     [3e+00, 3e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 12 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   5.000000e-01   0.000000e+00      0s
       1    5.7500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.750000000e+03
-----------------------------------------------
Optimal objective value: 5750.00 DKK
P_G[0]: 0.50 pu
theta[0]: 0.00 rad
P_G[1]: 1.50 pu
theta[1]: 0.1